In [36]:
my_api_key = '591e4342b8c0b235a6689a21d44aa57a'

In [37]:
from pprint import pprint
import requests #用於API HTTP requests
import json     #用於输入出json

url_api = "http://restapi.amap.com/v3/config/district"
parameters = {'keywords': '中国', 
              'subdistrict': 1,
              'key': "591e4342b8c0b235a6689a21d44aa57a"}
r = requests.get (url_api, params=parameters)
data = r.json()

In [38]:
print( { str(x['adcode']):x['name'] for x in data['districts'][0]['districts'] } )

{'110000': '北京市', '500000': '重庆市', '820000': '澳门特别行政区', '440000': '广东省', '350000': '福建省', '460000': '海南省', '140000': '山西省', '710000': '台湾省', '520000': '贵州省', '450000': '广西壮族自治区', '620000': '甘肃省', '410000': '河南省', '130000': '河北省', '340000': '安徽省', '430000': '湖南省', '310000': '上海市', '420000': '湖北省', '230000': '黑龙江省', '150000': '内蒙古自治区', '640000': '宁夏回族自治区', '360000': '江西省', '320000': '江苏省', '220000': '吉林省', '210000': '辽宁省', '370000': '山东省', '650000': '新疆维吾尔自治区', '120000': '天津市', '630000': '青海省', '610000': '陕西省', '540000': '西藏自治区', '510000': '四川省', '810000': '香港特别行政区', '530000': '云南省', '330000': '浙江省'}


In [39]:
places = { int(x['adcode']):x['name'] for x in data['districts'][0]['districts'] }
keywords = ['ZARA','H&M','UNIQLO','GAP','PULL&BEAR']

In [40]:
import requests #用於API HTTP requests
import pandas as pd


def amap_api_search_by_keyword_city_code(kw, cc, api_key):

    parameters = {'key': api_key}  # 高德地图API 获取Key
    parameters.update( {'keywords': kw,
                        'city':     cc,
                        'citylimit': True
                       } )

    pois = []   # 建构最终结果列表pois
    pg_no = 1   # 建构pg_no，每次给不一样的可选参数page值

    while True: # 不断迭代，直到 break跳出
        parameters.update({'page':pg_no})  #建构可选参数page值 为 pg_no

        r = requests.get ("http://restapi.amap.com/v3/place/text", params=parameters)
        data = r.json()
        #print(data)

        no_pois = int(data['count'])      # 计算 应有的实质数量

        if no_pois>0:
            pois.extend(data.get('pois', []))         # 使用Python列表之extend方法，把返回数据中的结果data['pois']存入pois
            no_pois_this_search = len(pois)   # 计算 累积实质结果数
            if (no_pois_this_search >= no_pois):  # 若 累积实质结果数 >= 应有的实质数量
                break                                   # 则结束迭代跳出
            else:
                pg_no +=1                               # 否则找下一页数据，把 pg_no增1

        else:
            break                                    # empty

    if len(pois)>0:
        # 使用pandas 模块处理数据
        df_input = pd.DataFrame(pois)

        # 选择部份栏位准备输出
        select_fields = ['location', 'address', 'adname', 'cityname', 'id', 'importance', 'name', 'pname', 'tel', 'type', 'typecode']
        df = df_input[select_fields]

        # 经纬度 增加新栏位字段 places 及 keywords
        df = df.assign( 经 = [x.split(',')[0] for x in df['location']])
        df = df.assign( 纬 = [x.split(',')[1] for x in df['location']])
        
        df = df.assign( places_code = [cc]*len(df['location']))
        df = df.assign( places = [places[cc]]*len(df['location']))
        df = df.assign( keywords = [kw]*len(df['location']))
        
    else:
        df = pd.DataFrame([])
    
    # 使用 return返回
    return(df)


In [41]:
{ c:places[c] for c in places.keys() if c in [810000, 820000]}

{810000: '香港特别行政区', 820000: '澳门特别行政区'}

In [42]:
import os.path
fn_output = "data_outcome.tsv"

for k in [keywords[0]]:  # keywords
    for p in { c:places[c] for c in places.keys() if c in [810000, 820000]}: # places
        d = amap_api_search_by_keyword_city_code(k, p, my_api_key)
        print ("读取 {地点} 中的 {关键字} 搜索结果...共{几个}个".format(地点=places[p], 关键字=k, 几个=len(d)))
        
        if os.path.exists(fn_output): 
            header_needed = False
        else:
            header_needed = True
        
        d.to_csv(fn_output, encoding='utf8', sep='\t', mode='a',
                                index = False,  index_label=False,
                                header = header_needed)

读取 澳门特别行政区 中的 ZARA 搜索结果...共3个
读取 香港特别行政区 中的 ZARA 搜索结果...共14个


In [43]:
df_all = pd.read_csv(fn_output, encoding='utf8', sep="\t")

In [44]:
import os.path
fn_output = "data_outcome.tsv"

for k in [keywords[0]]:  # keywords
    for p in { c:places[c] for c in places.keys() if c in [810000, 820000]}: # places
        d = amap_api_search_by_keyword_city_code(k, p, my_api_key)
        print ("读取 {地点} 中的 {关键字} 搜索结果...共{几个}个".format(地点=places[p], 关键字=k, 几个=len(d)))
        
        if os.path.exists(fn_output): 
            header_needed = False
        else:
            header_needed = True
        
        d.to_csv(fn_output, encoding='utf8', sep='\t', mode='a',
                                index = False,  index_label=False,
                                header = header_needed)

读取 澳门特别行政区 中的 ZARA 搜索结果...共3个
读取 香港特别行政区 中的 ZARA 搜索结果...共14个


In [45]:
# 全部执行
import os.path
fn_output = "data_outcome.tsv"

for k in keywords:
    for p in places:
        d = amap_api_search_by_keyword_city_code(k, p, my_api_key)
        print ("读取 {地点} 中的 {关键字} 搜索结果...共{几个}个".format(地点=places[p], 关键字=k, 几个=len(d)))

        if os.path.exists(fn_output): 
            header_needed = False
        else:
            header_needed = True
        
        # header  若数据档不存在则输出时包括栏位标题，若已存在则不包括
        d.to_csv(fn_output, encoding='utf8', sep='\t', mode='a',
                                index = False,  index_label=False,
                                header = header_needed)
        

读取 北京市 中的 ZARA 搜索结果...共26个
读取 重庆市 中的 ZARA 搜索结果...共8个
读取 澳门特别行政区 中的 ZARA 搜索结果...共3个
读取 广东省 中的 ZARA 搜索结果...共26个
读取 福建省 中的 ZARA 搜索结果...共12个
读取 海南省 中的 ZARA 搜索结果...共2个
读取 山西省 中的 ZARA 搜索结果...共4个
读取 台湾省 中的 ZARA 搜索结果...共4个
读取 贵州省 中的 ZARA 搜索结果...共1个
读取 广西壮族自治区 中的 ZARA 搜索结果...共4个
读取 甘肃省 中的 ZARA 搜索结果...共3个
读取 河南省 中的 ZARA 搜索结果...共9个
读取 河北省 中的 ZARA 搜索结果...共6个
读取 安徽省 中的 ZARA 搜索结果...共9个
读取 湖南省 中的 ZARA 搜索结果...共4个
读取 上海市 中的 ZARA 搜索结果...共19个
读取 湖北省 中的 ZARA 搜索结果...共6个
读取 黑龙江省 中的 ZARA 搜索结果...共9个
读取 内蒙古自治区 中的 ZARA 搜索结果...共5个
读取 宁夏回族自治区 中的 ZARA 搜索结果...共2个
读取 江西省 中的 ZARA 搜索结果...共4个
读取 江苏省 中的 ZARA 搜索结果...共29个
读取 吉林省 中的 ZARA 搜索结果...共4个
读取 辽宁省 中的 ZARA 搜索结果...共19个
读取 山东省 中的 ZARA 搜索结果...共16个
读取 新疆维吾尔自治区 中的 ZARA 搜索结果...共3个
读取 天津市 中的 ZARA 搜索结果...共7个
读取 青海省 中的 ZARA 搜索结果...共0个
读取 陕西省 中的 ZARA 搜索结果...共7个
读取 西藏自治区 中的 ZARA 搜索结果...共0个
读取 四川省 中的 ZARA 搜索结果...共19个
读取 香港特别行政区 中的 ZARA 搜索结果...共14个
读取 云南省 中的 ZARA 搜索结果...共8个
读取 浙江省 中的 ZARA 搜索结果...共25个
读取 北京市 中的 H&M 搜索结果...共23个
读取 重庆市 中的 H&M 搜索结果...共8个
读取 澳门特别行政区 中的 H&M 搜索结果...共0个

In [46]:
df_all = pd.read_csv(fn_output, encoding='utf8', sep="\t")

In [47]:
len(df_all)

1672

In [48]:
df_all.columns

Index(['location', 'address', 'adname', 'cityname', 'id', 'importance', 'name',
       'pname', 'tel', 'type', 'typecode', '经', '纬', 'places_code', 'places',
       'keywords'],
      dtype='object')

In [49]:
df_all.pivot_table(values='name', index='places', columns='keywords', 
                         aggfunc=lambda x: len(x.unique()))

keywords,GAP,H&M,PULL&BEAR,UNIQLO,ZARA
places,,,,,
上海市,29.0,18.0,6.0,68.0,18.0
云南省,5.0,3.0,1.0,13.0,8.0
内蒙古自治区,NaN,1.0,3.0,7.0,4.0
北京市,31.0,22.0,7.0,59.0,24.0
台湾省,3.0,NaN,NaN,7.0,3.0
吉林省,NaN,2.0,2.0,9.0,4.0
四川省,10.0,12.0,4.0,27.0,15.0
天津市,4.0,9.0,1.0,18.0,7.0
宁夏回族自治区,NaN,NaN,NaN,2.0,2.0
